In [1]:
import pandas as pd

test_label = pd.read_csv('./Temporal_Anomaly_Annotation_for_Testing_Videos.txt', sep='  ')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
test_label

,Videos,Classes,1_Start_Frame,1_End_Frame,2_Start_Frame,2_End_Frame
0,Abuse028_x264.mp4,Abuse,165,240,-1,-1
1,Abuse030_x264.mp4,Abuse,1275,1360,-1,-1
2,Arrest001_x264.mp4,Arrest,1185,1485,-1,-1
3,Arrest007_x264.mp4,Arrest,1530,2160,-1,-1
4,Arrest024_x264.mp4,Arrest,1005,3105,-1,-1
...,...,...,...,...,...,...
285,Vandalism007_x264.mp4,Vandalism,240,750,-1,-1
286,Vandalism015_x264.mp4,Vandalism,2010,2700,-1,-1
287,Vandalism017_x264.mp4,Vandalism,270,330,780,840
288,Vandalism028_x264.mp4,Vandalism,1830,1980,2400,2670


# 1. VideoFrameGenerator

#### Class(Fixed)
- Normal : 150
- Arson (+ Explosion) : 50 + 50 = 100
- Assault (+ Abuse, Fighting) : 50 + 50 + 50 = 150
- Burglary : 100

#### Image Size(Fixed)
- 128 X 128

#### \# of Frames(Fixed)
- 64

#### Data Split(Fixed)
- 7 : 2 : 1

#### Color Scale
- **Gray** or RGB

#### Frame Generator
- Basic or **Sliding** or OpticalFlow

In [10]:
!pip install keras-video-generators

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import glob
import keras

from keras_video import SlidingFrameGenerator

classes = [i.split(os.path.sep)[1] for i in glob.glob('zip_integrate/*')]
classes.sort()

SIZE = (128, 128)
CHANNELS = 1
NBFRAME = 64
BS = 1024

# pattern to get videos and classes
glob_pattern='zip_integrate/{classname}/*.mp4'

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

# Create sliding frame generator
train = SlidingFrameGenerator(
    classes=classes, # class list
    glob_pattern=glob_pattern, # directory path
    nb_frames=NBFRAME, # #of frames to return for each sequence
    rescale=1/255., # normalization
    split_val=.2, # split validation
    split_test=.1, # split test
    shuffle=True, # randomize
    batch_size=BS, # batch size
    target_shape=SIZE, # image size
    nb_channel=CHANNELS, # gray scale
    transformation=data_aug, # data augmentation
    use_frame_cache=False)
    
valid = train.get_validation_generator()
test = train.get_test_generator()

class Arson, validation count: 20, test count: 8, train count: 72
class Assault, validation count: 30, test count: 12, train count: 108
class Burglary, validation count: 20, test count: 8, train count: 72
class Normal_Videos-Part-1, validation count: 30, test count: 12, train count: 108
Total data: 4 classes for 360 files for train
Checking files to find possible sequences, please wait...
For 360 files, I found 23400 possible sequence samples
Total data: 4 classes for 100 files for validation
Checking files to find possible sequences, please wait...
For 100 files, I found 6500 possible sequence samples
Total data: 4 classes for 40 files for test
Checking files to find possible sequences, please wait...
For 40 files, I found 2600 possible sequence samples


In [4]:
import matplotlib.pyplot as plt
import keras_video.utils

keras_video.utils.show_sample(test, index=0, random=False, row_width=10, row_height=10)

KeyboardInterrupt: 

# 2. Modeling

#### Conv2d + GRU

In [5]:
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalAveragePooling2D

def build_convnet(shape=(128, 128, 3)):
    momentum = .9

    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape, padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalAveragePooling2D())
    return model

In [6]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(shape=(64, 128, 128, 3), nbout=4):
    # Create our convnet with (128, 128, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (64, 128, 128, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [7]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (64, 128, 128, 3)

model = action_model(INSHAPE, len(classes))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 64, 512)           4688064   
_________________________________________________________________
gru (GRU)                    (None, 64)                110976    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [8]:
optimizer = keras.optimizers.Adam(0.01)

model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

In [9]:
EPOCHS=50
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside

callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights_gray_sliding_conv_gru.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]

In [10]:
%%time

model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

KeyboardInterrupt: 

In [ ]:
model.evaluate(
    test)